In [3]:
import pandas as pd
import numpy as np 
from matplotlib import pyplot as plt

from sklearn.preprocessing import Normalizer
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE, RandomOverSampler
from sklearn.model_selection import train_test_split


import torch 
import torch.nn as nn 
from torch.utils.data import random_split, DataLoader, TensorDataset 
import torch.nn.functional as F
import torch.optim as optim

import flwr as fl
from flwr.common import Metrics

import sys

In [4]:
# import platform, cpuinfo, GPUtil, psutil
# print(f"OS: {platform.uname().system} {platform.uname().release}")
# print(f"CPU: {cpuinfo.get_cpu_info()['brand_raw']}")
# print(f"GPU: {GPUtil.getGPUs()[0].name}")
# print(f"Memory: {psutil.virtual_memory().total / (1024 ** 3):.2f} GB")

In [5]:
print("Python version:", sys.version)
print("Version info:", sys.version_info)

Python version: 3.11.4 (tags/v3.11.4:d2340ef, Jun  7 2023, 05:45:37) [MSC v.1934 64 bit (AMD64)]
Version info: sys.version_info(major=3, minor=11, micro=4, releaselevel='final', serial=0)


In [6]:
DEVICE = torch.device("cpu")  # Try "cuda" to train on GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)

Training on cpu using PyTorch 2.0.1+cpu and Flower 1.5.0


In [7]:
# X = dataset.drop('target', axis=1)
# y = dataset['target']
# X,y

# X_global_train, X_global_test, y_global_train, y_global_test = train_test_split(X, y, test_size=0.2, random_state=42)
# X_global_train, X_global_test, y_global_train, y_global_test

In [8]:
#DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [9]:
#data = pd.read_csv('./data.csv')

# class CustomDataset(Dataset):
#     def __init__(self, data, target):
#         self.data = data
#         self.target = target
# 
#     def __len__(self):
#         return len(self.data)
# 
#     def __getitem__(self, idx):
#         feature = torch.FloatTensor(self.data[idx, :])
#         target = torch.LongTensor(self.target[idx])
#         return feature, target

In [10]:
NUM_CLIENTS = 3

EPOCHS = 10
BATCH_SIZE = 10

IN_FEATURES = 3
HIDDEN_LAYERS = 128
OUT_FEATURES = 8

In [11]:
def load_datasets():
    df = pd.read_csv('./data.csv')
    df = df.rename(columns={'type': 'target'})
    normalize = ['temperature', 'pressure', 'humidity']
    scaler = Normalizer()
    df[normalize] = scaler.fit_transform(df[normalize])
    
    # X = df.iloc[:, :-1]
    # y = df.iloc[:, -1]
        
    split_ratio = 0.8
    split_index = int(len(df) * split_ratio)
    
    trainset = df.iloc[:split_index, :]
    testset = df.iloc[split_index:, :]
    print(trainset)
    print(testset)
    
load_datasets()


        temperature  pressure  humidity  target
622000     0.621362  0.012722  0.783420       0
218767     0.382035  0.011630  0.924075       0
331426     0.550945  0.211892  0.807194       0
218657     0.323645  0.014595  0.946066       0
164213     0.844971  0.101424  0.525107       0
...             ...       ...       ...     ...
187186     0.317109  0.014100  0.948284       0
625948     0.344829  0.237268  0.908183       0
416298     0.602934  0.161305  0.781314       0
172093     0.989371  0.022967  0.143589       0
345948     0.631181  0.010923  0.775559       0

[520193 rows x 4 columns]
        temperature  pressure  humidity  target
510953     0.776598  0.025113  0.629496       0
220580     0.950552  0.021499  0.309821       0
403981     0.331335  0.018495  0.943332       1
72939      0.842861  0.027093  0.537449       0
255290     0.366813  0.000155  0.930295       0
...             ...       ...       ...     ...
644167     0.594258 -0.132081  0.793355       0
259178     0.